# Python 201.3

## Nível intermediário em Python

Os notebooks dessa segunda etapa, focam especificamente em features intermediárias/avançadas da linguagem.

Tenha em mente que algumas questões apresentadas neste notebook, farão referência aos arquivos .py encontrados dentro do diretório src no mesmo nível.

### Paralelismo e Concorrência

A linguagem de programação python tem um histórico bem antigo relacioado a estruturas de processamento paralelo e concorrência.

Uma das grandes reclamações de muitos programadores da linguagem é a reconhecida [GIL (Global Interpreter Lock)](https://realpython.com/python-gil/). Diferente de outras linguagens que tem [Garbage Collector](https://en.wikipedia.org/wiki/Garbage_collection_(computer_science)), Python utiliza-se de outro modelo para verificar se as variáveis tem algum tipo de referência, chamado de [Reference Counting](https://en.wikipedia.org/wiki/Reference_counting).

> *The Python Global Interpreter Lock or GIL, in simple words, is a mutex (or a lock) that allows only one thread to hold the control of the Python interpreter.*

Dessa maneira o uso da linguagem acaba por impor algumas restrições na questão de paralelismo. Devido a restrição da GIL, todo programa em Python é executado em apenas uma única thread (nos mesmos moldes de javascript por exemplo). Podemos criar diversas threads em python, mas elas acabem por ser escalonadas para serem executadas uma por vez, tendo uma alternância entre elas.

![Threading](../../img/thread.png)

Neste sentido a linguagem acaba por implementar outro tipo de estrutura para a criação de paralelismo, ao invés de criarmos diversas threads, podemos criar diversos processos.

Uma das grandes diferenças é que não existe no caso dos processos memória compartilhada, evitando assim [Race Conditions](https://en.wikipedia.org/wiki/Race_condition). Outra diferença é que os processos encapsulam todo uma nova estrutura do interpretador python, ocasionando assim um gasto extra de processamento e memória.

> P.S.: Esta sendo implementado na versão 3.8 da linguagem, uma maneira de se compartilhar memória entre processos.

Apesar de tudo, ambas as threads e process tem sua valia... a regra de ouro é:
 - Caso seu programa faça uso intensido de processamento (CPU), utilize processos, pois ele escalonará melhor seu código, fazendo-o executar mais rápido.
 - Se, for utilizado I/O ou Network Access por exemplo, neste sentido a melhor opção são threads!
 - Assim, como o caso acima, veremos também o uso das keywords async/await que podem ser usadas para substituir o uso das threads.

#### concurrent.futures

Infelizmente em python existem algumas formas de se usar threads e processos (ver referências), mas nas últimas versões da linguagem foi criada uma estrutura bem simples e direta para a execução de código e é esta estrutura que vamos apresentar aqui.

In [1]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [2]:
import math

NUMBERS = range(0, 2)

def cpu_heavy(x):
    count = 0
    for i in range(x**2):
        count += i**2

def threading_example(fn=cpu_heavy, chunksize=100):
    """Threaded version: 8 threads."""
    items = []
    # Melhores práticas sugerem que max_workers para thread deve ser n = m * num_cores
    with ThreadPoolExecutor(max_workers=8) as executor:
        items = executor.map(fn, NUMBERS, chunksize=chunksize)
    return list(items)

def processing_example(fn=cpu_heavy, chunksize=100):
    """Multiprocessed version: 4 'threads'."""
    items = []
    # Melhores práticas sugerem que max_workers para process deve ser n = num_cores
    with ProcessPoolExecutor(max_workers=4) as executor:
        items = executor.map(fn, NUMBERS, chunksize=chunksize)
    return list(items)

def normal_example(fn=cpu_heavy):
    """Single threaded version."""
    items = []
    for i in NUMBERS:
        items.append(fn(i))
    return items

ex1, ex2, ex3 = threading_example(), processing_example(), normal_example()
print(ex1==ex2)
print(ex1==ex3)
print(ex2==ex3)

True
True
True


#### %timeit

No Jupyter Notebook podemos executar um método mágico chamado timeit (o mesmo pode ser executado via linha de comando para testar scripts em python).

Este método executa nosso código e retorna o tempo que levou para o mesmo ser executado. Vamos utilizar aqui para demonstrar em termos de performance de execução quais das 3 funções acabam por ter melhor performance.

In [3]:
NUMBERS = range(0, 250)

%timeit threading_example(fn=cpu_heavy)
%timeit processing_example(fn=cpu_heavy)
%timeit normal_example(fn=cpu_heavy)

3.55 s ± 642 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.24 s ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.13 s ± 50.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### I/O Bound

Caso nosso método de nome **cpu_heavy** tivesse muito I/O (como acesso a rede), neste caso o ideal é utilizar threading.

Para exemplificar, criamos o método **io_heavy**, o qual fará um HTTP GET (usando a biblioteca requests), e retorna verdadeiro ou falso caso a url seja encontrada.

Devido a peculiaridade da função **io_heavy**, o **chunksize** utilizado deve ser o padrão (o qual é 1)... isso porque caso contrário não dividiremos corretamente a carga, o que ocasionaria perda de performance, principalmente na versão com **ProcessPoolExecutor** (faça o teste, aumente o chunksize para 5 por exemplo).

In [4]:
import requests

def io_heavy(x):
    r = requests.get('https://google.com.br')
    if r.status_code == 200:
        return True
    return False

In [5]:
NUMBERS = range(0, 10)

%timeit threading_example(fn=io_heavy, chunksize=1)
%timeit processing_example(fn=io_heavy, chunksize=1)
%timeit normal_example(fn=io_heavy)

1.24 s ± 186 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.54 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.28 s ± 1.53 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### AsyncIO

Em versões mais modernas da linguagem Python, os desenvolvedores podem utilizar novas keywords para realizar a concorrência de suas aplicações. Em muitas outras linguagens esse conceito de Asynchronous I/O e Event Loop é bem conhecido.

Em versões novas da linguagem Javascript é possível utilizar praticamente as mesmas keywords e permitir que seu código se torne concorrente!

Para exemplificar o uso, vamos modificar nosso código io_heavy. Detalhe importante, que todo o código assíncrono usando as keywords async/await deve ser usando em processamento de rotinas que são I/O bound, nos mesmos moldes do módulo ThreadPoolExecutor, visto acima.

In [ ]:
import asyncio
import requests

async def aio_heavy(x):
    r = requests.get('https://google.com.br')
    await asyncio.sleep(1)
    if r.status_code == 200:
        return True
    return False

async def asyncio_example(NUMBERS):
    items = []
    for i in NUMBERS:
        item = await aio_heavy(i)
        items.append(item)
    return items

def main():
    NUMBERS = range(0, 10)
    loop = asyncio.get_event_loop()
    loop.create_task(asyncio_example(NUMBERS))

%timeit main()

3.33 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Como este código fora executado dentro do Jupyter Notebook, e o mesmo já possui um event loop sendo executado, foram utilizadas as linhas com as funções **get_event_loop** e **create_task**.

Para exemplificar de forma natural (caso não tivéssemos um event loop já sendo executado), acesse a 

### Referências
 - [Understanding the Python GIL](https://www.dabeaz.com/python/UnderstandingGIL.pdf)
 - [What is the Python Global Interpreter Lock (GIL)?](https://realpython.com/python-gil/)
 - [An Intro to Threading in Python](https://realpython.com/intro-to-python-threading/)
 - [Speed Up Your Python Program With Concurrency](https://realpython.com/python-concurrency/)
 - [Async IO in Python: A Complete Walkthrough](https://realpython.com/async-io-python/)
 - [threading - Manage concurrent threads](https://pymotw.com/3/threading/index.html)
 - [multiprocessing - Manage Processes Like Threads](https://pymotw.com/3/multiprocessing/index.html)
 - [asyncio - Asynchronous I/O, event loop, and concurrency tools](https://pymotw.com/3/asyncio/index.html)
 - [concurrent.futures - Manage Pools of Concurrent Tasks](https://pymotw.com/3/concurrent.futures/index.html)